In [1]:
import os
from dotenv import load_dotenv

import pandas as pd
import numpy as np
import datetime

import pyodbc
from sqlalchemy import create_engine

from faker import Faker
from faker.providers import internet
import random

load_dotenv()


True

In [53]:
fake = Faker(['pt_BR'])
fake.add_provider(internet)

from util.fake_generator import FakeGenerator
fakegen = FakeGenerator(fake)

date_now = datetime.datetime.now()

In [ ]:
file_name = 'FAKE.xlsx'
df = pd.read_excel(f'../data/processados/{file_name}')
df.head(5)

In [ ]:
qtd_linhas = len(df.index)
qtd_linhas

In [ ]:
coluna_ref = 'Created at'
df[coluna_ref] = pd.to_datetime(df[coluna_ref])

# df['ano'] = df[coluna_ref].dt.to_period('Y')
df['ano_mes'] = df[coluna_ref].dt.to_period('M')
distribuicao_mensal = df['ano_mes'].value_counts().sort_index()
distribuicao_mensal

In [ ]:
date_columns = []

for column in date_columns:
    new_column_name = 'delta_' + column
    df[new_column_name] = pd.to_datetime(df[column]) - df[coluna_ref]

    media_delta = df[new_column_name].mean()
    max_delta = df[new_column_name].max()
    min_delta = df[new_column_name].min()
    print(f'### Coluna: {column} ###')
    print(f"Média do tempo delta: {media_delta}")
    print(f"Máximo tempo delta: {max_delta}")
    print(f"Mínimo tempo delta: {min_delta} \n")

In [9]:
df.head(5)

,NEW_ID,ID,Created at,Data de Criação Ajustado,Finished at,Due date,Nome da empresa,Tempo de Conclusão,Creator,Current phase,Quantidade de itens baixados,ano_mes,delta_Data de Criação Ajustado,delta_Finished at,delta_Due date
0,204904985,554904985,2022-07-18 17:38:05.497,NaT,2022-07-18 18:45:05.764,NaN,Agrichem,NaN,Gisele de França Mello,Concluído,NaN,2022-07,NaT,0 days 01:07:00.267000,NaT
1,205979919,555979919,2022-07-20 19:32:23.510,NaT,2022-07-20 21:00:31.758,NaN,Agrichem,NaN,Gisele de França Mello,Concluído,NaN,2022-07,NaT,0 days 01:28:08.248000,NaT
2,207953107,557953107,2022-07-26 13:11:50.788,NaT,2022-07-26 14:06:05.285,NaN,Agrichem,NaN,Pipebot,Concluído,NaN,2022-07,NaT,0 days 00:54:14.497000,NaT
3,203501193,563501193,2022-08-09 16:51:56.537,NaT,NaT,NaN,Nutrien Útilfértil,NaN,Usuário Integração RPA Accenture - Pipefy,Descarte,NaN,2022-08,NaT,NaT,NaT
4,203529793,563529793,2022-08-09 17:52:57.708,NaT,NaT,NaN,Nutrien Útilfértil,NaN,Usuário Integração RPA Accenture - Pipefy,Descarte,NaN,2022-08,NaT,NaT,NaT


## Fake generate

In [24]:
random.seed(42)

columns = [
   
]

In [ ]:
df_filted = df[df['Current phase'] != 'Concluído']
seed_phase = df_filted['Current phase'].unique()
seed_phase

In [ ]:
created_date = fake.date_time_between(start_date=datetime.date(2020,1,1), end_date=datetime.date(2023,10,31))

end_date = random.choice([None, None,created_date + fakegen.gen_random_day_time(10), created_date + fakegen.gen_random_day_time(100)])

due_date = random.choice([created_date + datetime.timedelta(days=15), None])
print(due_date)
print(end_date)
print(end_date < due_date)



In [54]:
data = []
for _ in range(3000):
    id = f"20{str(fake.unique.random_int(min=1000000, max=9999999))}"
    
    created_date = fake.date_time_between(start_date=datetime.date(2020,1,1), end_date=datetime.date(2023,10,31))

    end_date = random.choice([None, None,created_date + fakegen.gen_random_day_time(10), created_date + fakegen.gen_random_day_time(100)])
    
    due_date = random.choice([created_date + datetime.timedelta(days=15), None])

    if due_date == None:
        in_due = random.choice([True, False]) 
    elif end_date != None: 
        in_due = end_date < due_date
    else:
        in_due = date_now < due_date
    
    
    company_name = fake.company()
    
    conclusion_time = ((end_date - created_date).days * 24 * 3600 + (end_date - created_date).seconds) / 3600 if end_date is not None else None    

    creator = random.choice(['Usuário', 'Chatbot', fake.ascii_company_email()])

    current_phase = 'Concluído' if end_date != None else random.choice(seed_phase)

    number_downloaded_items = random.choice([1, fake.random_int(min=1, max=110)])

    is_abroad = random.choice(['Nacional', 'Nacional', 'Internacional'])
    
    data.append([id, created_date, end_date, due_date, in_due, company_name, conclusion_time, creator,current_phase, number_downloaded_items, is_abroad])


In [ ]:
df_fake = pd.DataFrame(data, columns=columns)
df_fake.head(10)

In [56]:
year_current = date_now.year
month_current = date_now.month

In [57]:
df_fake = df_fake.reset_index(drop=True)
df_fake.to_excel(f'../data/{month_current}.{year_current}.xlsx', index=None)

In [58]:
df_purge = df_fake.head(int((len(df_fake.index)) * (fake.random_int(10,20) / 100)))
len(df_purge.index)

540

In [ ]:
df_purge = df_purge[['ID', 'Nome da empresa']].drop_duplicates()
df_purge.head(5)

In [62]:
df_purge.to_excel(f'../data/{month_current}.{year_current}.xlsx', index=None)